# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [2]:
import napari
from octopuslite import DaskOctopusLiteLoader
import glob
import numpy as np

# Load images

In [3]:
# define experiment ID and select a position
expt = 'ND0013'
pos = 'Pos1'
# point to where the data is
root_dir = '/home/nathan/data/kraken/ras'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_paths = glob.glob(f'{root_dir}/{expt}/{pos}/*.npy')
print(transform_paths)
#tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'

['/home/nathan/data/kraken/ras/ND0013/Pos1/mask_reversed_transform_tensor.npy', '/home/nathan/data/kraken/ras/ND0013/Pos1/mask_reversed_clipped_transform_tensor.npy', '/home/nathan/data/kraken/ras/ND0013/Pos1/gfp_transform_tensor.npy', '/home/nathan/data/kraken/ras/ND0013/Pos1/gfp_transform_tensor_clipped.npy']


In [4]:
transform_path = transform_paths[1] #'/home/nathan/data/kraken/ras/ND0019/Pos2/mask_reversed_clipped_transform_tensor.npy'

In [5]:
# lazily load images
images = DaskOctopusLiteLoader(image_path, 
                              # transforms = transform_path,
                               crop = (1200,1600),
                               remove_background = True)
images.channels

Using cropping: (1200, 1600)


[<Channels.BRIGHTFIELD: 0>,
 <Channels.GFP: 1>,
 <Channels.RFP: 2>,
 <Channels.IRFP: 3>,
 <Channels.MASK_2CH: 99>]

In [6]:
gfp = images['gfp']
rfp = images['rfp']
irfp = images['irfp']
# ras_masks = images['mask_gfp'].astype(int)
# nuc_masks = images['mask'].astype(int)

### Add tracks

In [ ]:
import btrack
from btrack.utils import tracks_to_napari

#### iRFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_1'
                                  ) as h:
    irfp_tracks = h.tracks

In [ ]:
irfp_tracks, irfp_properties, irfp_graph = tracks_to_napari(irfp_tracks, ndim = 2)

#### RFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_2'
                                  ) as h:
    rfp_tracks = h.tracks

In [ ]:
rfp_tracks, rfp_properties, rfp_graph = tracks_to_napari(rfp_tracks, ndim = 2)

# Launch viewer

In [6]:
N = 1200 ## number of frames to load

In [7]:
viewer = napari.Viewer()
viewer.add_image(gfp[0:1200].astype(np.uint8), name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp[0:1200].astype(np.uint8), name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(irfp[0:1200].astype(np.uint8), name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#
# viewer.add_labels(nuc_masks[0:1200].astype(int), name = 'masks',
# #                 contrast_limits=[0,255],
#                blending = 'additive',)
# viewer.add_labels(ras_masks[0:1200].astype(int), name = 'cyto masks',
# #                 contrast_limits=[0,255],
#                blending = 'additive',)
# viewer.add_tracks(irfp_tracks)
# viewer.add_tracks(rfp_tracks)

Assistant skips harvesting pyclesperanto as it's not installed.


__init__.py (165): Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.


<Image layer 'irfp' at 0x7f37eddb62e0>

# Ras mask tidier

In [113]:
ras_masks.shape

(219, 1200, 1600)

In [ ]:
for cell_ID, class_label in enumerate(details['class_id'], 1):
    #needs erosion step to stop merging of labels
    segment = nd.binary_erosion(labels==cell_ID)
    mask[segment] = class_label
    # set background to zero
    mask[labels == 0] = 0